In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import pickle
import sys
from itertools import product
from time import time

import numpy as np
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC

sys.path.append('/home/share/libraries/bdpy/')
import bdpy
from bdpy.preproc import select_top
from bdpy.ml import add_bias,make_cvindex
from bdpy.stats import corrcoef
sys.path.append('/home/shirakawa/kspy/')
import cal_mean_ci



# Nakamura

In [8]:
v_spec = np.load("ventricle_spectrum.npy")
a_spec = np.load("article_spectrum.npy")
# list for sampling features
NUM_FEATURES = [i for i in range(1, 100)] + [i * 50 for i in range(2, 45)]

In [9]:
# ventricle: 0, article: 1
# choose 71 samples from vertricle and make test data and label data
def make_dataset(v_spec, a_spec):
    """
    choose 71 samples from vertricle and make test data and label data
    """
    N = a_spec.shape[0]
    choice = np.random.choice(v_spec.shape[0], N)
    v_spec_random = v_spec[choice]
    data = np.vstack((v_spec_random, a_spec))
    # create dataset and labels(v: 0, a:1)
    label = label = np.array([0] * N + [1] * N)
    return data, label

In [10]:
def cross_validation(data, label, n=10):
    """
    return list of train_data, test_data, train_label, test_label
    return
        list: list of shape(n, 4)
    """
    index = np.arange(data.shape[0])
    index = index % n
    cvindex = make_cvindex(index)
    cvindex = np.array(cvindex)
    cross_valid = []
    for i in range(cvindex.shape[2]):
        train_data = data[cvindex[0][:, i] == True]
        test_data = data[cvindex[1][:, i] == True]
        train_label = label[cvindex[0][:, i] == True]
        test_label = label[cvindex[1][:, i] == True]
        cross_valid.append((train_data, test_data, train_label, test_label))
    return cross_valid

In [11]:
# use linar SVM and calculate the score
def classify(train_data, test_data, train_label, test_label, n_features):
    # scale data
    scaler = StandardScaler()
    scaler.fit(train_data)
    scaled_train_data = scaler.transform(train_data)
    scaled_test_data = scaler.transform(test_data)
    # train classifier
    clf = LinearSVC()
    # use n_features
    clf.fit(scaled_train_data[:, : n_features], train_label)
    # get score
    score = clf.score(scaled_test_data[:, :n_features], test_label)
    return score

In [12]:
data, label = make_dataset(v_spec, a_spec)

# Shirakawa

In [2]:
# loading data
atrial = pd.read_csv("Atrial.csv", header=None) # (time *  cell)
ventricle = pd.read_csv("Ventricle.csv",header = None) # time * cell)
# transposing for training
atrial = atrial.T
ventricle = ventricle.T
# get number of samples 
[N,c] = np.shape(atrial)
[M,C] = np.shape(ventricle)

aFPow = np.zeros(4500/2)
vFPow = np.zeros(4500/2)
featnum = len(aFPow)
# get Power Spectrum
for i in range(1,N):
    
    adata = atrial.iloc[i][3:].astype(float)
    
    F = np.fft.fft(adata)
    aF = np.abs(F)**2
    aF = aF[:featnum]
    aFPow = np.vstack([aFPow,aF])
aFPow = aFPow[1:,:]    

for i in range(1,M):
    
    vdata = ventricle.iloc[i][3:].astype(float)
    
    F = np.fft.fft(vdata)
    vF = np.abs(F)**2
    vF = vF[:featnum]
    vFPow = np.vstack([vFPow,vF])
vFPow = vFPow[1:,:]    


# get minimum for chance level is 50 % (-1 means datas' index exclude)
n = np.min([N-1,M-1])

In [6]:
vFPow - v_spec

array([[  5.96046448e-08,   0.00000000e+00,  -7.27595761e-12, ...,
         -7.27595761e-12,  -7.27595761e-12,  -1.81898940e-11],
       [  0.00000000e+00,   1.45519152e-11,  -1.74622983e-10, ...,
         -1.81898940e-11,  -2.13162821e-13,   2.18278728e-11],
       [  0.00000000e+00,   7.27595761e-12,  -4.36557457e-11, ...,
          3.63797881e-12,  -9.09494702e-12,  -1.73372428e-12],
       ..., 
       [  0.00000000e+00,  -1.09139364e-11,   0.00000000e+00, ...,
          8.52651283e-14,  -4.54747351e-13,  -3.41060513e-13],
       [  0.00000000e+00,  -2.72848411e-12,   0.00000000e+00, ...,
         -1.36424205e-12,   2.72848411e-12,  -1.81898940e-12],
       [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
         -3.26849658e-13,   1.08357767e-13,   5.68434189e-14]])

In [7]:
aFPow - a_spec

array([[  0.00000000e+00,   5.45696821e-12,  -2.91038305e-11, ...,
         -7.27595761e-12,  -1.45519152e-11,   1.45519152e-11],
       [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
          7.27595761e-12,   5.09317033e-11,   3.27418093e-11],
       [  0.00000000e+00,  -2.91038305e-11,   0.00000000e+00, ...,
          0.00000000e+00,  -5.00222086e-12,   2.72848411e-12],
       ..., 
       [  0.00000000e+00,  -1.09139364e-11,   2.18278728e-11, ...,
         -9.09494702e-13,  -2.50111043e-12,  -9.09494702e-13],
       [  0.00000000e+00,   0.00000000e+00,   1.45519152e-11, ...,
          4.43378667e-12,   5.25801624e-13,  -2.55795385e-13],
       [  0.00000000e+00,   0.00000000e+00,  -9.09494702e-13, ...,
          5.68434189e-13,   2.41584530e-13,  -6.11066753e-13]])